In [ ]:
! pip install pytorch_pretrained_bert

     |████████████████████████████████| 133kB 43.3MB/s 


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from pytorch_pretrained_bert import BertModel
from torch.autograd import Variable

class bigru_attention(nn.Module):
    def __init__(self, bert_config, tagset_size, embedding_dim, hidden_dim, rnn_layers, dropout_ratio, dropout1, use_cuda):
        super(bigru_attention, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.rnn_layers = rnn_layers
        self.word_embeds = BertModel.from_pretrained(bert_config)
        # 双向GRU，//操作为了与后面的Attention操作维度匹配，hidden_dim要取偶数！
        self.bigru = nn.GRU(embedding_dim, hidden_dim, num_layers=rnn_layers, bidirectional=True,dropout=dropout_ratio, batch_first=True)
        self.dropout1 = nn.Dropout(p=dropout1)
        # 由nn.Parameter定义的变量都为requires_grad=True状态
        self.weight_W = nn.Parameter(torch.Tensor(hidden_dim*2, hidden_dim*2))
        self.weight_proj = nn.Parameter(torch.Tensor(hidden_dim*2, 1))
        self.fc = nn.Linear(hidden_dim*2,tagset_size)
        nn.init.uniform_(self.weight_W, -0.1, 0.1)
        nn.init.uniform_(self.weight_proj, -0.1, 0.1)
        self.use_cuda =  use_cuda

    def rand_init_hidden(self, batch_size):
        if self.use_cuda:
            return Variable(
                torch.randn(2 * self.rnn_layers, batch_size, self.hidden_dim)).cuda(), Variable(
                torch.randn(2 * self.rnn_layers, batch_size, self.hidden_dim)).cuda()
        else:
            return Variable(
                torch.randn(2 * self.rnn_layers, batch_size, self.hidden_dim)), Variable(
                torch.randn(2 * self.rnn_layers, batch_size, self.hidden_dim))
 
    def forward(self, sentence, attention_mask=None):
        batch_size = sentence.size(0)
        seq_length = sentence.size(1)
        embeds, _ = self.word_embeds(sentence, attention_mask=attention_mask, output_all_encoded_layers=False)
        hidden = self.rand_init_hidden(batch_size)
        gru_out, hiden = self.bigru(embeds) # [seq_len, bs, hid_dim]
        d_gru_out = self.dropout1(gru_out)
        x = d_gru_out
        # # # Attention过程，与上图中三个公式对应
        u = torch.tanh(torch.matmul(x, self.weight_W))
        att = torch.matmul(u, self.weight_proj)
        att_score = F.softmax(att, dim=1)
        scored_x = x * att_score
        # # # Attention过程结束
        feat = torch.sum(scored_x, dim=1)
        y = self.fc(feat)
        return y


备用

In [ ]:
class InputFeatures(object):
    def __init__(self, text, label, input_id, input_mask):
        self.text = text
        self.label = label
        self.input_id = input_id
        self.input_mask = input_mask

def load_vocab(vocab_file):
    vocab = {}
    index = 0
    with open(vocab_file, "r", encoding="utf-8") as reader:
        while True:
            token = reader.readline()
            if not token:
                break
            token = token.strip()
            vocab[token] = index
            index += 1
    return vocab
#修改
def load_file(file_path):
    contents = open(file_path, encoding='utf-8').readlines()
    texts = []
    labels = []
    for line in contents:
        if line != '\n':
            line = line.strip().split(' ')
            texts.append([char for char in line[-1]])
            labels.append(int(line[0]))#目前不确定要不要加【】
    return texts, labels
 
def load_data(file_path, max_length, vocab):
    texts, labels = load_file(file_path)
    assert len(texts) == len(labels)
    result = []
    for i in range(len(texts)):
        token = texts[i]
        label = int(labels[i])
        if len(token) > max_length-2:
            token = token[0:(max_length-2)]
        tokens_f =['[CLS]'] + token + ['[SEP]']
        input_ids = [int(vocab[i]) if i in vocab else int(vocab['[UNK]']) for i in tokens_f]
        mask_bool=1
        input_mask = [mask_bool] * len(input_ids)
        while len(input_ids) < max_length:
            input_ids.append(0)
            input_mask.append(0)
        assert len(input_ids) == max_length
        assert len(input_mask) == max_length
        #assert len(label_ids) == max_length 实体识别标签序列用
        feature = InputFeatures(text=tokens_f, label=label, input_id=input_ids, input_mask=input_mask)
        result.append(feature)
    return result

In [ ]:
train_file='/content/drive/My Drive/Colab Notebooks/kashgari/ka re/Copy of train.txt'
dev_file='/content/drive/My Drive/Colab Notebooks/kashgari/ka re/Copy of test.txt'
max_length=100
vocab_file='/content/drive/My Drive/chinese_roberta_wwm_ext_pytorch.zip_files/vocab.txt'
batch_size=1
tagset_size = 16
epochs=50

In [ ]:
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report
if torch.cuda.is_available():
    device = torch.device("cuda", 0)
    print('device',device)
    use_cuda = True
else:
    device = torch.device("cpu")
    use_cuda = False
vocab = load_vocab(vocab_file)
train_data = load_data(train_file, max_length=max_length, vocab=vocab)
train_ids = torch.LongTensor([temp.input_id for temp in train_data])
train_masks = torch.LongTensor([temp.input_mask for temp in train_data])
train_tags = torch.LongTensor([temp.label for temp in train_data])
train_dataset = TensorDataset(train_ids, train_masks, train_tags)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

dev_data = load_data(dev_file, max_length=max_length, vocab=vocab)
dev_ids = torch.LongTensor([temp.input_id for temp in dev_data])
dev_masks = torch.LongTensor([temp.input_mask for temp in dev_data])
dev_tags = torch.LongTensor([temp.label for temp in dev_data])
dev_dataset = TensorDataset(dev_ids, dev_masks, dev_tags)
dev_loader = DataLoader(dev_dataset, shuffle=True, batch_size=batch_size)

#def evaluate(model, dev_loader):
#    model.eval()
 #   pred = []
#    for i, train_batch in enumerate(dev_loader):
#        model.zero_grad()
 #       sentence, masks, tags = train_batch
 #       sentence, masks, tags = Variable(sentence), Variable(masks), Variable(tags)
 #       if use_cuda:
 #           sentence = sentence.cuda()
 #           asks = masks.cuda()
 #           tags = tags.cuda()
 ##       predict_tags = F.softmax(model(sentence)).tolist()
  #      pred_tag=predict_tags.index(max(predict_tags))
  #      pred.extend([pred_tag])
 #       print(classification_report(dev_tags, pred))
        
model = bigru_attention('/content/drive/My Drive/chinese_roberta_wwm_ext_pytorch.zip_files', tagset_size, 768, 200, 1,
                      dropout_ratio=0.4, dropout1=0.4, use_cuda = use_cuda)
if use_cuda:
    model.cuda()
model.train()
losser=torch.nn.CrossEntropyLoss()
optimizer = getattr(optim, 'Adam')
optimizer = optimizer(model.parameters(), lr=0.00003, weight_decay=0.00005)
best_f = -100
for epoch in range(epochs):
    print('epoch: {}trrain'.format(epoch))
    for i, train_batch in enumerate(tqdm(train_loader)):
        model.train()
        model.zero_grad()
        sentence, masks, tags = train_batch
        sentence, masks, tags = Variable(sentence), Variable(masks), Variable(tags)
        if use_cuda:
            sentence = sentence.cuda()
            masks = masks.cuda()
            tags = tags.cuda()
        loss = losser(model(sentence),tags)
        loss.backward()
        optimizer.step()
    print('epoch: {}loss: {}'.format(epoch, loss.item()))
    model.eval()
    pred = []
    true=[]
    for i, dev_batch in enumerate(dev_loader):
        model.zero_grad()
        sentence, masks, tags = dev_batch
        sentence, masks, tags = Variable(sentence), Variable(masks), Variable(tags)
        if use_cuda:
            sentence = sentence.cuda()
            asks = masks.cuda()
            tags = tags.cuda()
        predict_tags = F.softmax(model(sentence)).tolist()
        pred_tag=predict_tags[0].index(max(predict_tags[0]))
        pred.extend([pred_tag])
        true.extend(tags.tolist())
    print(classification_report(true, pred))

device cuda:0


  0%|          | 1/1112 [00:00<03:19,  5.57it/s]

epoch: 0trrain


100%|██████████| 1112/1112 [02:58<00:00,  6.22it/s]


epoch: 0loss: 2.37180495262146


...

epoch: 9loss: 0.4652981758117676


  0%|          | 1/1112 [00:00<02:28,  7.47it/s]

              precision    recall  f1-score   support

           0       0.91      0.84      0.87        98
           1       0.94      0.94      0.94        18
           2       0.74      0.77      0.76        22
           3       0.89      0.80      0.84        10
           4       0.84      0.95      0.89        57
           5       1.00      0.88      0.94        25
           6       0.86      1.00      0.92         6
           7       0.67      1.00      0.80         2
           8       0.67      0.67      0.67         9
           9       0.25      1.00      0.40         2
          10       0.55      0.50      0.52        12
          11       0.50      0.29      0.36         7
          12       0.67      0.67      0.67         3
          13       0.67      0.80      0.73         5
          14       1.00      1.00      1.00         1
          15       0.00      0.00      0.00         1

    accuracy                           0.83       278
   macro avg       0.70   

  7%|▋         | 82/1112 [00:13<02:43,  6.30it/s]

KeyboardInterrupt: ignored

In [ ]:
print(classification_report(true, pred))

              precision    recall  f1-score   support

           0       0.91      0.84      0.87        98
           1       0.94      0.94      0.94        18
           2       0.74      0.77      0.76        22
           3       0.89      0.80      0.84        10
           4       0.84      0.95      0.89        57
           5       1.00      0.88      0.94        25
           6       0.86      1.00      0.92         6
           7       0.67      1.00      0.80         2
           8       0.67      0.67      0.67         9
           9       0.25      1.00      0.40         2
          10       0.55      0.50      0.52        12
          11       0.50      0.29      0.36         7
          12       0.67      0.67      0.67         3
          13       0.67      0.80      0.73         5
          14       1.00      1.00      1.00         1
          15       0.00      0.00      0.00         1

    accuracy                           0.83       278
   macro avg       0.70   

In [ ]:
model_name = model_name = '/content/drive/My Drive/chinese_roberta_wwm_ext_pytorch.zip_files/' + str(epochs) + ".pkl"
torch.save(model.state_dict(), model_name)

In [ ]:
import torch
model =bigru_attention('/content/drive/My Drive/chinese_roberta_wwm_ext_pytorch.zip_files', tagset_size, 768, 200, 1,
                      dropout_ratio=0.4, dropout1=0.4, use_cuda = True)
model.load_state_dict(torch.load('/content/drive/My Drive/Colab Notebooks/kashgari/ka re/50.pkl'))
model.cuda()

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.4 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


bigru_attention(
  (word_embeds): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
          

In [ ]:

import pandas as pd
import json
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
df=pd.read_csv("/content/drive/My Drive/Colab Notebooks/bert-bilstm-crf-master/bert_lstm_crf_pytorch/data/012.txt")
df.columns=['关系','类型1','人物1','类型2','人物2','文本']  
relations = list(df['关系'].unique())
texts = []
for per1, per2, text in zip(df['人物1'].tolist(), df['人物2'].tolist(), df['文本'].tolist()):
    text = '$'.join([str(per1), str(per2), text.replace(str(per1), len(str(per1))*'#').replace(str(per2), len(str(per2))*'#')])
    texts.append(text)
df['text'] = texts
tokens=[]
ids=[]
vocab = load_vocab(vocab_file)
for i in range(len(texts)):
    token = texts[i]
    if len(token) > max_length-2:
        token = token[0:(max_length-2)]
    tokens_f =['[CLS]'] + list(token) + ['[SEP]']
    tokens.append(tokens_f)
    input_ids = [int(vocab[i]) if i in vocab else int(vocab['[UNK]']) for i in tokens_f]
    ids.append(input_ids)
    while len(input_ids) < max_length:
        input_ids.append(0)
pred_ids = torch.LongTensor([temp for temp in ids])
pred_dataset = TensorDataset(pred_ids)
pred_loader = DataLoader(pred_dataset, shuffle=False, batch_size=batch_size)
model.eval()
pred = []
for i, pred_batch in enumerate(pred_loader):
    model.zero_grad()
    sentence = pred_batch
    sentence= Variable(sentence[0])
    if True:
        sentence = sentence.cuda()
    predict_tags = F.softmax(model(sentence)).tolist()
    pred_tag=predict_tags[0].index(max(predict_tags[0]))
    pred.extend([pred_tag])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
pred

[4,
 4,
 0,
 0,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 6,
 4,
 4,
 4,
 2,
 2,
 0,
 0,
 6,
 6,
 14,
 0,
 6,
 6,
 0,
 6,
 6,
 6,
 6,
 0,
 1,
 1,
 2,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 0,
 5,
 5,
 5,
 1,
 0,
 5,
 5,
 5,
 1,
 5,
 5,
 5,
 1,
 0,
 0,
 4,
 0,
 4,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 5,
 1,
 1,
 4,
 6,
 4,
 4,
 6,
 4,
 4,
 4,
 4,
 4,
 2,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 0,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 0,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 0,
 0,
 4,
 0,
 4,
 4,
 0,
 4,
 0,
 4,
 0,
 4,
 0,
 4,
 0,
 0,
 4,
 6,
 4,
 0,
 6,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 5,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 10,
 10,
 10,
 4,
 0,
 0,
 4,
 0,
 4,
 4,
 2,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,


In [ ]:
raw=df.values.tolist()

In [ ]:
raw[:5]

[[0,
  '医疗指标',
  '因子',
  '医疗指标',
  'NGF',
  '目的:研究黄皮提取物对星形胶质细胞分泌神经营养因子(NGF)的影响',
  '因子$NGF$目的:研究黄皮提取物对星形胶质细胞分泌神经营养##(###)的影响'],
 [1,
  '诊断技术',
  'RT-PCR法',
  '医疗指标',
  'NGF',
  '方法:RT-PCR法测定给予10μmol/L的黄皮提取物不同时间点后星形胶质细胞内NGF和低亲和力受体P75的mRNA水平,同时ELISA方法验证培养上清中NGF的分泌水平',
  'RT-PCR法$NGF$方法:#######测定给予10μmol/L的黄皮提取物不同时间点后星形胶质细胞内###和低亲和力受体P75的mRNA水平,同时ELISA方法验证培养上清中###的分泌水平'],
 [2,
  '诊断技术',
  'RT-PCR法',
  '医疗指标',
  'P75',
  '方法:RT-PCR法测定给予10μmol/L的黄皮提取物不同时间点后星形胶质细胞内NGF和低亲和力受体P75的mRNA水平,同时ELISA方法验证培养上清中NGF的分泌水平',
  'RT-PCR法$P75$方法:#######测定给予10μmol/L的黄皮提取物不同时间点后星形胶质细胞内NGF和低亲和力受体###的mRNA水平,同时ELISA方法验证培养上清中NGF的分泌水平'],
 [3,
  '诊断技术',
  'RT-PCR法',
  '医疗指标',
  'mRNA',
  '方法:RT-PCR法测定给予10μmol/L的黄皮提取物不同时间点后星形胶质细胞内NGF和低亲和力受体P75的mRNA水平,同时ELISA方法验证培养上清中NGF的分泌水平',
  'RT-PCR法$mRNA$方法:#######测定给予10μmol/L的黄皮提取物不同时间点后星形胶质细胞内NGF和低亲和力受体P75的####水平,同时ELISA方法验证培养上清中NGF的分泌水平'],
 [4,
  '诊断技术',
  'RT-PCR法',
  '诊断技术',
  'ELISA',
  '方法:RT-PCR法测定给予10μmol/L的黄皮提取物不同时间点后星形胶质细胞内NGF和低亲和力受体P75的mRNA水平,同时ELISA方法验证培养上清中NGF的分泌水平

In [ ]:
predicted_sen=list(zip(raw,pred))

In [ ]:
predicted_sen[:5]

[([0,
   '医疗指标',
   '因子',
   '医疗指标',
   'NGF',
   '目的:研究黄皮提取物对星形胶质细胞分泌神经营养因子(NGF)的影响',
   '因子$NGF$目的:研究黄皮提取物对星形胶质细胞分泌神经营养##(###)的影响'],
  4),
 ([1,
   '诊断技术',
   'RT-PCR法',
   '医疗指标',
   'NGF',
   '方法:RT-PCR法测定给予10μmol/L的黄皮提取物不同时间点后星形胶质细胞内NGF和低亲和力受体P75的mRNA水平,同时ELISA方法验证培养上清中NGF的分泌水平',
   'RT-PCR法$NGF$方法:#######测定给予10μmol/L的黄皮提取物不同时间点后星形胶质细胞内###和低亲和力受体P75的mRNA水平,同时ELISA方法验证培养上清中###的分泌水平'],
  4),
 ([2,
   '诊断技术',
   'RT-PCR法',
   '医疗指标',
   'P75',
   '方法:RT-PCR法测定给予10μmol/L的黄皮提取物不同时间点后星形胶质细胞内NGF和低亲和力受体P75的mRNA水平,同时ELISA方法验证培养上清中NGF的分泌水平',
   'RT-PCR法$P75$方法:#######测定给予10μmol/L的黄皮提取物不同时间点后星形胶质细胞内NGF和低亲和力受体###的mRNA水平,同时ELISA方法验证培养上清中NGF的分泌水平'],
  0),
 ([3,
   '诊断技术',
   'RT-PCR法',
   '医疗指标',
   'mRNA',
   '方法:RT-PCR法测定给予10μmol/L的黄皮提取物不同时间点后星形胶质细胞内NGF和低亲和力受体P75的mRNA水平,同时ELISA方法验证培养上清中NGF的分泌水平',
   'RT-PCR法$mRNA$方法:#######测定给予10μmol/L的黄皮提取物不同时间点后星形胶质细胞内NGF和低亲和力受体P75的####水平,同时ELISA方法验证培养上清中NGF的分泌水平'],
  0),
 ([4,
   '诊断技术',
   'RT-PCR法',
   '诊断技术',
   'ELISA',
   '方法:RT-PCR法测定给予10μmol/L的黄皮提取物

In [ ]:
rel_dic={
 0:"unknown",
 1:"临床表现",
 2:"药物治疗",
 3:"预防",
 4:"病因",
 5:"同义",
 6:"病理",
 7:"护理",
 8:"诊断",
 9:"报告",
 10:"非药治疗",
 11:"观察",
 12:"检测项目为",
 13:"作用于",
 14:"导致",
 15:"分为"
}

In [ ]:
pred_result=[]
for i in pred:
    pred_result.append(rel_dic[i])

In [ ]:
pred_result

['病因',
 '病因',
 'unknown',
 'unknown',
 '病因',
 '病因',
 '病因',
 '病因',
 '病因',
 '病因',
 '病因',
 '病因',
 '病因',
 '病因',
 '病因',
 '病因',
 '病因',
 '病因',
 '病因',
 '病因',
 '病因',
 '病因',
 '病因',
 '病因',
 '病因',
 '病因',
 '病因',
 '病因',
 '病因',
 '病理',
 '病因',
 '病因',
 '病因',
 '药物治疗',
 '药物治疗',
 'unknown',
 'unknown',
 '病理',
 '病理',
 '导致',
 'unknown',
 '病理',
 '病理',
 'unknown',
 '病理',
 '病理',
 '病理',
 '病理',
 'unknown',
 '临床表现',
 '临床表现',
 '药物治疗',
 '病因',
 '病因',
 '病因',
 '病因',
 '病因',
 '病因',
 '病因',
 'unknown',
 '同义',
 '同义',
 '同义',
 '临床表现',
 'unknown',
 '同义',
 '同义',
 '同义',
 '临床表现',
 '同义',
 '同义',
 '同义',
 '临床表现',
 'unknown',
 'unknown',
 '病因',
 'unknown',
 '病因',
 '病因',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 'unknown',
 '临床表现',
 '临床表现',
 '同义',
 '临床表现',
 '临床表现',
 '病因',
 '病理',
 '病因',
 '病因',
 '病理',
 '病因',
 '病因',
 '病因',
 '病因',
 '病因',
 '药物治疗',
 '病因',
 '病因',
 '病因',
 '病因',
 '病因',
 '病因',
 '病因',
 '病因',
 '病因',
 '病因',
 '病因',
 '病因',
 '病因',
 '病因',
 '病因',
 '病因',
 '病因',
 '病因',
 '病因',
 '病因',
 '病因',
 '病因',
 '病因',
 '病因',
 '病因',
 '

In [ ]:
predicted_sen=list(zip(raw,pred_result))